Make sure to pip install pickle, praw, and imdbpy

In [4]:
import json
import requests
import praw
import pprint
from collections import Counter
import pickle
import os
import shutil
import datetime
from textblob import TextBlob

In [ ]:
All pickled objects

In [ ]:
with open( "pickled/movies_data.p", "rb" ) as f:
    movies_data = pickle.load(f)
    
with open( "pickled/subreddit_test.p", "rb" ) as f:
    data = pickle.load(f)
    



1/2

Get a dict of popular movies from 2010 to 2015. 100 from each year, 

** NOTE: not nessesarily movies that were released in that year. **

In [13]:
import tmdbsimple as tmdb
with open( "API_keys/tmdb_api_key.txt", "rb" ) as f:
     tmdb_api_key = f.read()

tmdb.API_KEY = tmdb_api_key.strip()
discover = tmdb.Discover()

all_responses = {}
for n in range(2010,2016):
    responses= []
    for x in range(1,6):
        response = discover.movie(page=x, year=n)
        responses.append(response['results'])    

    # flatten list of lists
    responses = [val for sublist in responses for val in sublist]
    all_responses[n] = responses
    

2/2

Get and pickle data from movie dict

** NOTE: not nessesarily movies that were released in that year. **



In [7]:
import time
# get all dicts together
aggregate_responses = [all_responses[year_of_movies] for year_of_movies in all_responses]

# flatten
aggregate_responses = [ movie for yearlist in aggregate_responses for movie in yearlist]
all_title_dates = [(movie['title'], movie['release_date']) for movie in aggregate_responses]

# aggregate_responses = [tmdb.Movies(movie['id']).info() for movie in aggregate_responses]
movie = tmdb.Movies(all_responses[2010][0]['id']).info()

movies_data = []
counter = 0
end_count = len(aggregate_responses)
for movie in aggregate_responses:
    movie_data = tmdb.Movies(movie['id']).info()
    movies_data.append(movie_data)
    time.sleep(1)
    counter += 1
    if(counter % 10 == 0):
        print(counter, end_count)

# extended information on all movies (not keyed on by year)
with open( "pickled/movies_data.p", "wb" ) as f:
    pickle.dump(movies_data, f)
movies_data

10 600
20 600
30 600
40 600
50 600
60 600
70 600
80 600
90 600
100 600
110 600
120 600
130 600
140 600
150 600
160 600
170 600
180 600
190 600
200 600
210 600
220 600
230 600
240 600
250 600
260 600
270 600
280 600
290 600
300 600
310 600
320 600
330 600
340 600
350 600
360 600
370 600
380 600
390 600
400 600
410 600
420 600
430 600
440 600
450 600
460 600
470 600
480 600
490 600
500 600
510 600
520 600
530 600
540 600
550 600
560 600
570 600
580 600
590 600
600 600


/home/connor/miniconda2/lib/python3.5/site-packages/IPython/core/formatters.py:90: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/home/connor/miniconda2/lib/python3.5/site-packages/IPython/core/formatters.py:96: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
/home/connor/miniconda2/lib/python3.5/site-packages/IPython/core/formatters.py:675: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
/home/connor/miniconda2/lib/python3.5/site-packages/IPython/core/formatters.py:667: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
/home/connor/miniconda2/lib/python3.5/site-packages/IP

[{'adult': False,
  'backdrop_path': '/5XPPB44RQGfkBrbJxmtdndKz05n.jpg',
  'belongs_to_collection': {'backdrop_path': '/9s4BM48NweGFrIRE6haIul0YJ9f.jpg',
   'id': 87096,
   'name': 'Avatar Collection',
   'poster_path': '/nslJVsO58Etqkk17oXMuVK4gNOF.jpg'},
  'budget': 237000000,
  'genres': [{'id': 28, 'name': 'Action'},
   {'id': 12, 'name': 'Adventure'},
   {'id': 14, 'name': 'Fantasy'},
   {'id': 878, 'name': 'Science Fiction'}],
  'homepage': 'http://www.avatarmovie.com/',
  'id': 19995,
  'imdb_id': 'tt0499549',
  'original_language': 'en',
  'original_title': 'Avatar',
  'overview': 'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.',
  'popularity': 8.473271,
  'poster_path': '/tcqb9NHdw9SWs2a88KCDD4V8sVR.jpg',
  'production_companies': [{'id': 25, 'name': '20th Century Fox'},
   {'id': 289, 'name': 'Ingenious Film Partners'},
   {'id': 444, 'name': 'Dune

Get most popular subreddits for respective movie.

In [5]:
# get top 5 subreddits for given movie
def get_top_subs(title):
    r = praw.Reddit(user_agent='reddit_hype')
    deadpool = r.search(title, limit=None)
    subreddits = Counter()
    try:
        for x in range(100):
            entry = next(deadpool)
            subreddit = entry.subreddit
            subreddits[subreddit] += 1
    except:
        pass

    return [x[0] for x in subreddits.most_common(5)]

In [6]:
# test
get_top_subs('deadpool')

/home/connor/miniconda2/lib/python3.5/site-packages/IPython/core/formatters.py:90: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/home/connor/miniconda2/lib/python3.5/site-packages/IPython/core/formatters.py:96: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
/home/connor/miniconda2/lib/python3.5/site-packages/IPython/core/formatters.py:675: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
/home/connor/miniconda2/lib/python3.5/site-packages/IPython/core/formatters.py:667: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
/home/connor/miniconda2/lib/python3.5/site-packages/IP

[Subreddit(subreddit_name='deadpool'),
 Subreddit(subreddit_name='movies'),
 Subreddit(subreddit_name='whowouldwin'),
 Subreddit(subreddit_name='Showerthoughts'),
 Subreddit(subreddit_name='OutOfTheLoop')]

Search function that returns a generator of submission objects.

In [7]:
# one year range of dates
# release_date in format 'yyyy-mm-dd'
def search_subreddit(subreddit, release_date):
    date = release_date.split('-')
    date = [int(x) for x in date]
    date = datetime.datetime(date[0], date[1], date[2])
    begin = str((date - datetime.timedelta(days=365)).timestamp()).split('.')[0]
    fin = str(date.timestamp()).split('.')[0]
    search_str = 'timestamp:' + begin + '..' + fin
    return subreddit.search(search_str, syntax='cloudsearch', limit=None)


In [8]:
# tests
r = praw.Reddit(user_agent='reddit_hype')
subreddits = r.search_reddit_names('deadpool')
subreddit = subreddits[0]
print(subreddit.title)
results = search_subreddit(subreddit, '2016-02-14')

print(next(results))
print(next(results))
print(next(results))
print(next(results))

Mr. Wade Winston Wilson
3894 :: No title needed
714 :: After all the trickery happening on this sub...
399 :: He truly is the best animal tamer.
1261 :: Deadpool just updated its Facebook cover photo. Looks like a nice rom...


Search function that returns a generator of submission objects matching a keyword.

In [9]:
# one year range of dates
# release_date in format 'yyyy-mm-dd'
def search_subreddit_keyword(subreddit, movie, release_date):
    date = release_date.split('-')
    date = [int(x) for x in date]
    date = datetime.datetime(date[0], date[1], date[2])
    begin = str((date - datetime.timedelta(days=365)).timestamp()).split('.')[0]
    fin = str(date.timestamp()).split('.')[0]
    search_str = 'timestamp:' + begin + '..' + fin
    keyword =  "title:'" + movie + "'" 
    search_str = "(and " + search_str + " " + keyword + ")"
    return subreddit.search(search_str, syntax='cloudsearch', limit=None)


In [10]:
# tests
r = praw.Reddit(user_agent='reddit_hype')
subreddits = r.search_reddit_names('deadpool')
subreddit = subreddits[0]
print(subreddit.title)
results = search_subreddit_keyword(subreddit, 'deadpool', '2016-02-14')

Mr. Wade Winston Wilson


In place pickling of submissions. Not being used, but could be depending on needs.

In [335]:
# may want to not pickle

#return customized name for pickle file
def getPickleFilename(subreddit, counter):
    return "pickled/" + str(subreddit) + "/" + str(counter) + ".p"

#The input must be a praw.objects.Subreddit instance.
#This function takes all of the posts (in the form of Submission objects) of a subreddit 
#and places them in lists of lengths at most 1000. Then, it pickles those lists. This allows
#for easy access to the data from Reddit so that you never have to have the entire dataset
#in memory all at once (which may cause overflow errors).
def pickleRedditData (subreddit, release_date):
    #create the generator for the Submission objects. The empty string is for an empty query to
    #include all posts in the results
    results = search_subreddit(subreddit, release_date)
    
    #counts how many pickle files we have written so far
    counter = 0
    
    submissions = []
    dir_name = "pickled/" + str(subreddit)
    if(os.path.isdir(dir_name)):
        shutil.rmtree(dir_name)
    os.mkdir(dir_name)
    try:
        while(True):
            for i in range(1000):
                submission = next(results)
                submissions.append(submission)
            filename = getPickleFilename(subreddit, counter)
            with open( filename, "wb" ) as f:
                pickle.dump(submissions, f)
                counter = counter + 1
    except StopIteration:
        filename = getPickleFilename(subreddit, counter)
        with open( filename, "wb" ) as f:
            pickle.dump(submissions, f)
            counter = counter + 1
        
    return submissions

    

In [336]:
# test
r = praw.Reddit(user_agent='reddit_hype')
subreddits = get_top_subs('deadpool')
subreddit = subreddits[0]
pickleRedditData(subreddit, '2016-02-12')
filename = getPickleFilename(subreddit, 0)
with open (filename, "rb") as f:
    #remember these results are just the submissions from one pickle file
    #There may be more pickle files.
    results = pickle.load(f)
    for i in range(10):
        print(results[i])

timestamp:1423728000..1455264000
3890 :: No title needed
497 :: Ryan Reynolds thanks fans for getting Deadpool made
605 :: I was on tinder, and saw this
544 :: Best Freddy Krueger make-up ever. (Hawkeye vs. Deadpool #1) (xpost r/c...
787 :: One of the many reasons why I love Deadpool - (Deadpool: Dracula's Gau...
451 :: And then they DIEEEEE
326 :: Deadpool official poster
355 :: Deadpool converse
792 :: Marvel uploaded this to their Facebook page
299 :: I like Deadpool and all, but I'm not drinking that.


Get 1000 submssions from a year before the release date from the given subreddit.

In [11]:
def redditData (subreddit, release_date):
    #create the generator for the Submission objects. The empty string is for an empty query to
    #include all posts in the results
    results = search_subreddit(subreddit, release_date)
    
    submissions = []
    try:
        for i in range(1000):
            submission = next(results)
            submissions.append(submission)
    except:
        pass
        
        
    return submissions

In [12]:
# test
r = praw.Reddit(user_agent='reddit_hype')
subreddits = get_top_subs('deadpool')
subreddit = subreddits[0]
subs = redditData(subreddit, '2016-02-12')
for i in range(10):
    print(subs[i])

3889 :: No title needed
713 :: After all the trickery happening on this sub...
397 :: He truly is the best animal tamer.
287 :: Real life cake-day, my wife is awesome guys. She would love some feedb...
476 :: "Deadpool" movie character's bio released.
595 :: Dear tax returns, thanks for being a thing
760 :: Deadpool | Trailer Trailer [HD] | 20th Century FOX
347 :: Reposting is bad, alright? I made MY stupid drawing a bit nicer.
410 :: i saw this on a doctor who page this morning.
565 :: So, this is happening.


Get 1000 submssions from a year before the release date from the given subreddit that relate to the given movie.

In [13]:
def redditKeywordData (subreddit, release_date, title):
    #create the generator for the Submission objects. The empty string is for an empty query to
    #include all posts in the results
    results = search_subreddit_keyword(subreddit, title, release_date)
    submissions = []
    try:
        for i in range(1000):
            submission = next(results)
            submissions.append(submission)
    except:
        pass
        
        
    return submissions

In [14]:
# test
r = praw.Reddit(user_agent='reddit_hype')
subreddits = get_top_subs('deadpool')
subreddit = subreddits[0]
subs = redditKeywordData(subreddit, '2016-02-12', 'deadpool')
for i in range(10):
    print(subs[i])

789 :: One of the many reasons why I love Deadpool - (Deadpool: Dracula's Gau...
1956 :: IT'S HERE! DEADPOOL TRAILER!
1654 :: Fantastic Four 2 May Be Dropped For Deadpool 2
1664 :: The only person the official Deadpool film twitter account follows is...
1331 :: My wife took a shot at drawing Deadpool. What do you guys think?
318 :: Feb 10 - DEADPOOL MOVIE DISCUSSION - Deadpool Release Discussion (Merc...
228 :: Deadpool V Deadpool: Dawn of Deadpool
905 :: Deadpool cameo idea x-post from r/tumblr
815 :: First official still from 'Deadpool'
760 :: Deadpool | Trailer Trailer [HD] | 20th Century FOX


In [17]:
class Movie_Data:
    def __init__(self, title, release_date, rating, budget, revenue):
        self.title = title
        self.release_date = release_date
        self.rating = rating
        self.budget = budget
        self.revenue = revenue
        if(budget == 0):
            self.budget_ratio = revenue
        else:
            self.budget_ratio = revenue/budget
        self.subreddits = []
        
    def add_subreddit(self, subreddit):
        self.subreddits = self.subreddits + [subreddit]
    
    def add_subreddits(self, subreddits):
        self.subreddits = self.subreddits + subreddits

class Subreddit_Data:
    def __init__(self, name, rel_upvotes, irrel_upvotes, post_scores):
        self.name = name
        self.rel_upvotes = rel_upvotes
        self.irrel_upvotes = irrel_upvotes
        self.post_scores = post_scores
    
    @property
    def score(self):
        # multiply by max score of 2 if v relevant    
        upvotes = self.rel_upvotes + self.irrel_upvotes
        return self.post_scores * (1 + (self.rel_upvotes/upvotes))
    


In [18]:
# test

test_movie = Movie_Data("Raja Gone Wild", "1995-04-04",100,10000,1000000)
print(test_movie.budget_ratio)

test_sub = Subreddit_Data("Crazy Profs caught in the act", 100, 2, 67364)
test_sub.score

100.0


133407.13725490196

Take in the post, get all upvotes, get scores, check posts

In [27]:
def get_post_data(post, release_date):
    release_date = release_date.split('-')
    release_date = [int(x) for x in release_date]
    release_date = datetime.datetime(release_date[0], release_date[1], release_date[2])
#   Killed run time, but can use below line to get more info to parse from submissions
#   subm = r.get_submission(submission_id = post.id)
    ups = post.ups
    date = datetime.datetime.fromtimestamp((post.created))
    score = 1.5 * (365 - ((release_date - date).days))
    return (ups, score)

In [20]:
# test

get_post_data(r.get_submission(submission_id = "468qqi"), '2016-2-12')


(3492, 556.5)

In [19]:
# test

subs = redditData(subreddit, '2016-02-12')
subs2 = redditKeywordData(subreddit, '2016-02-12', 'deadpool')

In [20]:
# test

release_date = '2016-02-12'
movie = 'deadpool'
count = 0
lens = len(subs)
total_ups = 0

print('Starting')
for sub in subs:
    if(count == 10):
        break
    (sub_ups, sub_score) = get_post_data(sub, release_date)
    total_ups += sub_ups
    count+=1
    print(count, lens, sub.title)
    
print('over halfway there')    
count = 0
rel_ups = 0
rel_score = 0
lens = len(subs2)
for sub in subs2:
    if(count == 10):
        break
    (sub_ups, sub_score) = get_post_data(sub, release_date)
    rel_ups += sub_ups
    rel_score += sub_score  
    count +=1
    print(count, lens, sub.title)

#(self, name, rel_upvotes, irrel_upvotes, post_scores)
# total ups - rel_ups may be negative depending on the size of the subs/ subs2
s = Subreddit_Data(subreddit.fullname, rel_ups, total_ups-rel_ups, rel_score)


    


Starting
1 987 No title needed
2 987 Ryan Reynolds thanks fans for getting Deadpool made
3 987 Breaking the Fourth Wall already?
4 987 Best Freddy Krueger make-up ever. (Hawkeye vs. Deadpool #1) (xpost r/comicbooks)
5 987 One of the many reasons why I love Deadpool - (Deadpool: Dracula's Gauntlet)
6 987 And then they DIEEEEE
7 987 Deadpool official poster
8 987 Deadpool converse
9 987 Marvel uploaded this to their Facebook page
10 987 I like Deadpool and all, but I'm not drinking that.
over halfway there
1 987 One of the many reasons why I love Deadpool - (Deadpool: Dracula's Gauntlet)
2 987 IT'S HERE! DEADPOOL TRAILER!
3 987 Fantastic Four 2 May Be Dropped For Deadpool 2
4 987 The only person the official Deadpool film twitter account follows is hello kitty
5 987 My wife took a shot at drawing Deadpool. What do you guys think?
6 987 Deadpool V Deadpool: Dawn of Deadpool
7 987 Feb 10 - DEADPOOL MOVIE DISCUSSION - Deadpool Release Discussion (Mercs for Money #1/Uncanny Avengers #5/Spide

In [21]:
# test

s.irrel_upvotes

-2000

Given a subreddit, release_date, and movie: return the corresponding Subreddit object.

In [26]:
def get_subreddit_info(subreddit, release_date, movie):
    subs = redditData(subreddit, release_date)
    count = 0
    lens = len(subs)
    total_ups = 0
    
    for sub in subs:
        (sub_ups, sub_score) = get_post_data(sub, release_date)
        total_ups += sub_ups
        count+=1

    count = 0
    rel_ups = 0
    rel_score = 0
    subs = redditKeywordData(subreddit, release_date, movie)
    lens = len(subs)
    for sub in subs:
        (sub_ups, sub_score) = get_post_data(sub, release_date)
        rel_ups += sub_ups
        rel_score += sub_score  
        count +=1
    
    #(self, name, rel_upvotes, irrel_upvotes, post_scores)
    return Subreddit_Data(subreddit.fullname, rel_ups, total_ups-rel_ups, rel_score)
    

Test and pickle dead subreddit subreddit data.

In [25]:
# test

data = get_subreddit_info(subreddit, '2016-02-12', 'deadpool')
with open( "pickled/subreddit_test.p", "wb" ) as f:
    pickle.dump(data, f)

In [36]:
# test

data.score

472653.64802255546

Get all 5 subreddits data and push it into the corresponding Movie object.

In [24]:
def get_subreddits_info(release_date, title, Movie):
    subreddits = get_top_subs(title)
    for sub in subreddits:
        #release date is str
        Movie.add_subreddit(get_subreddit_info(sub, release_date, title))
    return Movie

Create a movie object for each movie in db and return list of movie objects

In [23]:
def create_movies(movie_data):
    movie_list = []
    for movie in movie_data:
        # (self, title, release_date, rating, budget, revenue)    
        m = Movie_Data(movie['original_title'], movie['release_date'], movie['popularity'], movie['budget'], movie['revenue'])
        movie_list.append(m)
        
    return movie_list

In [ ]:
# test

create_movies(movies_data)[0].budget_ratio

Add corresponding subreddit objects to movie objects

In [28]:
def create_subreddits(movies):
    movie_list = []
    counter = 0
    for movie in movies:
        movie_list.append(get_subreddits_info(movie.release_date, movie.title, movie))
        print(counter, '600')
        counter += 1
        if counter == 20:
            return movie_list
    return movie_list

Create movie struct: a list of complete Movie objects 

In [17]:
m  = create_movies(movies_data)
movies_struct = create_subreddits(m)
with open( "pickled/movies_obj_data.p", "wb" ) as f:
    pickle.dump(movies_struct, f)

0 600
1 600
2 600
3 600
4 600
5 600
6 600
7 600
8 600
9 600
10 600
11 600
12 600
13 600
14 600
15 600
16 600
17 600
18 600
19 600


In [ ]:
with open( "pickled/movies_obj_data.p", "rb" ) as f:
    movies_struct = pickle.load(f)

In [30]:
# test
movies_struct

Create graphs of data

In [67]:
def combined_movie_hype(Movie):
    total_hype = 0
    for sub in Movie.subreddits:
        total_hype += sub.post_scores
    return total_hype

def combined_upvote_ratio(Movie):
    total_rel = 0
    total_irrel = 0
    for sub in Movie.subreddits:
        total_rel += sub.rel_upvotes
        total_irrel += sub.irrel_upvotes
    if total_irrel == 0:
        return 0
    return (total_rel/total_irrel+total_rel)

def subreddit_movie_hype(Movie):
    hype_list = []
    for sub in Movie.subreddits:
        hype_list.append(sub.post_scores)
    return hype_list

def all_movie_hype(Movies):
    total_hype = []
    for Movie in Movies:
        total_hype.append((str(Movie.revenue/Movie.budget), combined_movie_hype(Movie), Movie.title))
    return total_hype

all_movie_hype(movies_struct)

[('11.7363115907173', 0, 'Avatar'),
 ('5.159375', 26904.0, 'Inception'),
 ('1.856765005', 32863.5, 'Terminator Salvation'),
 ('40.844401833333336', 0, 'The Godfather'),
 ('3.6850524375', 2373.0, 'Shutter Island'),
 ('3.6571416153846155', 0, 'The Godfather: Part II'),
 ('13.122017685714285', 6814.5, 'The Hangover'),
 ('3.817223472', 2055.0, 'Harry Potter and the Deathly Hallows: Part 1'),
 ('25.215671615384615', 11860.5, 'Black Swan'),
 ('5.822540877777778', 0, 'Sherlock Holmes'),
 ('3.4353179642857143', 29650.5, 'Kick-Ass'),
 ('7.027320366666666', 20602.5, 'District 9'),
 ('7.877014275362319', 1461.0, 'Despicable Me'),
 ('6.8834415571428575', 0, 'Saving Private Ryan'),
 ('3.119666655', 19759.5, 'Iron Man 2'),
 ('4.338617881355932', 11754.0, 'Zombieland'),
 ('12.0652912', 0, 'Toy Story'),
 ('3.430879925', 11208.0, 'The Expendables'),
 ('15.09402212', 0, 'Beauty and the Beast'),
 ('5.623007875', 20676.0, 'The Social Network')]

In [49]:
import matplotlib.pyplot as plt
brr = []
h_score = []
for x in all_movie_hype(movies_struct):
    if x[1] != 0:
        brr.append(x[0])
        h_score.append(x[1])
plt.plot(brr, h_score, 'ro')
plt.show()


In [50]:
def all_movie_hype_upvotes(Movies):
    total_hype = []
    for Movie in Movies:
        total_hype.append((Movie.rating, combined_movie_hype(Movie), Movie.title))
    return total_hype

In [53]:
import matplotlib.pyplot as plt
brr = []
h_score = []
for x in all_movie_hype_upvotes(movies_struct):
    if x[1] != 0:
        brr.append(x[0])
        h_score.append(x[1])
plt.plot(brr, h_score, 'ro')
plt.show()


In [60]:
import matplotlib.pyplot as plt
brr = []
for x in movies_struct:
#     enter title here
    if 'wan' in x.title:
        subs_hype = subreddit_movie_hype(x)
        plt.plot(subs_hype, 'ro')
        plt.show()

In [70]:
import matplotlib.pyplot as plt
import statistics as s
stdevs = []
brr = []
for x in movies_struct:
    subs_hype = subreddit_movie_hype(x)
    stdevs.append(s.stdev(subs_hype))
    brr.append((x.revenue/x.budget))
        
plt.plot(stdevs, brr, 'ro')
plt.show()

In [68]:
# Relevant upvote ratio. Not looking so good. Probably need more than 1000 submissions
import matplotlib.pyplot as plt
uv_rat = []
popularity = []
for Movie in movies_struct:
    x = combined_upvote_ratio(Movie)
    if x == 0:
        continue
    uv_rat.append(x)
    popularity.append(Movie.rating)
plt.plot(uv_rat, popularity, 'ro')
plt.show()